In [120]:
import pandas as pd
from itertools import groupby
from operator import itemgetter

In [8]:
df = pd.read_csv('../rawdata/user_profiles_participation.csv')

In [9]:
df.columns = ['Years', 'Years Label', 'index', 'url']
df = df[['Years', 'url']]
df.head()

,Years,url
0,NaN,http://nanowrimo.org/participants/nicaless
1,2014 winner,http://nanowrimo.org/participants/nicaless
2,2015 participant,http://nanowrimo.org/participants/nicaless
3,2005 winner,http://nanowrimo.org/participants/rachel-b-moore
4,2006 winner,http://nanowrimo.org/participants/rachel-b-moore


In [25]:
years_new_list = []
for i in df['Years']:
    if isinstance(i, float):
        years_new_list.append("donor")
    else:
        years_new_list.append(i.split(" "))
        

In [42]:
years_new_list


['donor',
 ['2014', 'winner'],
 ['2015', 'participant'],
 ['2005', 'winner'],
 ['2006', 'winner'],
 'donor',
 ['2007', 'winner'],
 'donor',
 ['2008', 'winner'],
 'donor',
 ['2009', 'winner'],
 'donor',
 ['2010', 'winner'],
 'donor',
 ['2011', 'winner'],
 'donor',
 ['2012', 'participant'],
 'donor',
 ['2013', 'participant'],
 'donor',
 ['2014', 'winner'],
 'donor',
 ['2015', 'winner'],
 'donor',
 ['2015', 'participant'],
 ['2004', 'winner'],
 ['2005', 'winner'],
 ['2006', 'participant'],
 'donor',
 ['2007', 'participant'],
 'donor',
 ['2008', 'winner'],
 'donor',
 ['2009', 'participant'],
 ['2010', 'participant'],
 'donor',
 ['2011', 'winner'],
 'donor',
 ['2012', 'winner'],
 'donor',
 ['2013', 'winner'],
 'donor',
 ['2014', 'winner'],
 'donor',
 ['2015', 'winner'],
 ['2012', 'participant'],
 ['2014', 'participant'],
 ['2015', 'participant'],
 ['2012', 'participant'],
 ['2013', 'participant'],
 ['2014', 'participant'],
 ['2015', 'participant'],
 'donor',
 ['2015', 'winner'],
 ['2015', '

In [43]:
df['Years'] = years_new_list

In [45]:
writernames = pd.read_csv('../clean data/user_summary.csv')

In [49]:
writernames = writernames[['Writer Name', 'url']]
writernames.head()

,Writer Name,url
0,Nicaless,http://nanowrimo.org/participants/nicaless
1,Rachel B. Moore,http://nanowrimo.org/participants/rachel-b-moore
2,abookishbabe,http://nanowrimo.org/participants/abookishbabe
3,alexabexis,http://nanowrimo.org/participants/alexabexis
4,AllYellowFlowers,http://nanowrimo.org/participants/AllYellowFlo...


In [55]:
new_df = pd.merge(writernames, df, on='url')
new_df = new_df[['Writer Name', 'Years']]
new_df.head()

,Writer Name,Years
0,Nicaless,donor
1,Nicaless,"[2014, winner]"
2,Nicaless,"[2015, participant]"
3,Rachel B. Moore,"[2005, winner]"
4,Rachel B. Moore,"[2006, winner]"


In [59]:
allnames = writernames[['Writer Name']]
allnames.head()

,Writer Name
0,Nicaless
1,Rachel B. Moore
2,abookishbabe
3,alexabexis
4,AllYellowFlowers


In [63]:
groupnames = new_df.groupby('Writer Name')

In [136]:
list_dics = []
test = ["alexabexis"]
#for name in test:
for name in allnames['Writer Name']:
    group = groupnames.get_group(name)
    dic = {}
    donor_years = []
    win_years = []
    part_years = []
    
    for i in group.index:
        if group['Years'][i] == 'donor':
            donor_years.append(group['Years'][i+1][0])
        elif group['Years'][i][1] == "winner":
            win_years.append(group['Years'][i][0])
            part_years.append(group['Years'][i][0])
        else:
            part_years.append(group['Years'][i][0])
    
    donor_years = map(int, donor_years)
    consec_donor = 0
    for k, g in groupby(enumerate(donor_years), lambda (i, x): i-x):
        consec_donor = consec_donor + len(map(itemgetter(1), g))
        
    win_years = map(int, win_years)
    consec_wins = 0
    for k, g in groupby(enumerate(win_years), lambda (i, x): i-x):
        consec_wins = consec_wins + len(map(itemgetter(1), g))
    
    part_years = map(int, part_years)
    consec_part = 0
    for k, g in groupby(enumerate(part_years), lambda (i, x): i-x):
        consec_part = consec_part + len(map(itemgetter(1), g))
    
    dic['Writer Name'] = name
    dic['Consecutive Donor'] = consec_donor
    dic['Consecutive Wins'] = consec_wins
    dic['Consecutive Part'] = consec_part
    dic['Donor Years'] = donor_years
    dic['Win Years'] = win_years
    dic['Part Years'] = part_years
    list_dics.append(dic)

In [137]:
final_df = pd.DataFrame(list_dics)
final_df.head()

,Consecutive Donor,Consecutive Part,Consecutive Wins,Donor Years,Part Years,Win Years,Writer Name
0,1,2,1,[2014],"[2014, 2015]",[2014],Nicaless
1,9,11,9,"[2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...",Rachel B. Moore
2,1,1,0,[2015],[2015],[],abookishbabe
3,8,12,8,"[2007, 2008, 2009, 2011, 2012, 2013, 2014, 2015]","[2004, 2005, 2006, 2007, 2008, 2009, 2010, 201...","[2004, 2005, 2008, 2011, 2012, 2013, 2014, 2015]",alexabexis
4,0,3,0,[],"[2012, 2014, 2015]",[],AllYellowFlowers


In [138]:
final_df = final_df[['Writer Name', 'Consecutive Donor', 'Consecutive Wins', 'Consecutive Part', 'Part Years', 'Win Years', "Donor Years"]]

In [139]:
final_df.to_csv("../clean data/user_participation_clean.csv")

In [143]:
summary = pd.read_csv("../clean data/user_summary.csv")
del summary['Unnamed: 0']
summary.head()

,Writer Name,Member Length,LifetimeWordCount,url,Age,Birthday,Favorite books or authors,Favorite noveling music,Hobbies,Location,Occupation,Primary Role,Sponsorship URL
0,Nicaless,2,75148,http://nanowrimo.org/participants/nicaless,24,December 20,"Ursula Le Guin, J.K.","Classical, Musicals","Reading, Video Games, Blogging, Learning","San Francisco, CA",Consultant,NaN,NaN
1,Rachel B. Moore,10,528693,http://nanowrimo.org/participants/rachel-b-moore,NaN,NaN,"2666, Unaccustomed Earth, Exit Music, Crazy Lo...","Belle and Sebastian, Elliott Smith, PJ Harvey,...","Reading, volunteering, knitting, listening to ...",San Francisco,Writer/Office Monkey,Municipal Liaison,NaN
2,abookishbabe,1,29299,http://nanowrimo.org/participants/abookishbabe,NaN,April 2,"Colleen Hoover, Veronica Roth, Jennifer Niven,...",Tori Kelley,"Reading (DUH), Day dreaming, Going to Disneyla...","Sacramento, CA",Optometry Assistant,NaN,NaN
3,alexabexis,11,554300,http://nanowrimo.org/participants/alexabexis,NaN,NaN,NaN,Three Goddesses playlist Florence + the Machin...,"drawing, reading, movies & TV shows, comics, p...",New York City,NaN,Municipal Liaison,https://www.classy.org/fundraise?fcid=498118
4,AllYellowFlowers,3,41428,http://nanowrimo.org/participants/AllYellowFlo...,NaN,NaN,"Lolita, Jesus' Son, Ask the",the sound of the coffeemaker,cryptozoology,Allston,Bookstore Employee,NaN,NaN


In [144]:
summary = pd.merge(summary, final_df, on='Writer Name')

In [145]:
summary.head()

,Writer Name,Member Length,LifetimeWordCount,url,Age,Birthday,Favorite books or authors,Favorite noveling music,Hobbies,Location,Occupation,Primary Role,Sponsorship URL,Consecutive Donor,Consecutive Wins,Consecutive Part,Part Years,Win Years,Donor Years
0,Nicaless,2,75148,http://nanowrimo.org/participants/nicaless,24,December 20,"Ursula Le Guin, J.K.","Classical, Musicals","Reading, Video Games, Blogging, Learning","San Francisco, CA",Consultant,NaN,NaN,1,1,2,"[2014, 2015]",[2014],[2014]
1,Rachel B. Moore,10,528693,http://nanowrimo.org/participants/rachel-b-moore,NaN,NaN,"2666, Unaccustomed Earth, Exit Music, Crazy Lo...","Belle and Sebastian, Elliott Smith, PJ Harvey,...","Reading, volunteering, knitting, listening to ...",San Francisco,Writer/Office Monkey,Municipal Liaison,NaN,9,9,11,"[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...","[2007, 2008, 2009, 2010, 2011, 2012, 2013, 201..."
2,abookishbabe,1,29299,http://nanowrimo.org/participants/abookishbabe,NaN,April 2,"Colleen Hoover, Veronica Roth, Jennifer Niven,...",Tori Kelley,"Reading (DUH), Day dreaming, Going to Disneyla...","Sacramento, CA",Optometry Assistant,NaN,NaN,1,0,1,[2015],[],[2015]
3,alexabexis,11,554300,http://nanowrimo.org/participants/alexabexis,NaN,NaN,NaN,Three Goddesses playlist Florence + the Machin...,"drawing, reading, movies & TV shows, comics, p...",New York City,NaN,Municipal Liaison,https://www.classy.org/fundraise?fcid=498118,8,8,12,"[2004, 2005, 2006, 2007, 2008, 2009, 2010, 201...","[2004, 2005, 2008, 2011, 2012, 2013, 2014, 2015]","[2007, 2008, 2009, 2011, 2012, 2013, 2014, 2015]"
4,AllYellowFlowers,3,41428,http://nanowrimo.org/participants/AllYellowFlo...,NaN,NaN,"Lolita, Jesus' Son, Ask the",the sound of the coffeemaker,cryptozoology,Allston,Bookstore Employee,NaN,NaN,0,0,3,"[2012, 2014, 2015]",[],[]


In [146]:
summary.to_csv("../clean data/user_summary.csv")

In [147]:
summary

,Writer Name,Member Length,LifetimeWordCount,url,Age,Birthday,Favorite books or authors,Favorite noveling music,Hobbies,Location,Occupation,Primary Role,Sponsorship URL,Consecutive Donor,Consecutive Wins,Consecutive Part,Part Years,Win Years,Donor Years
0,Nicaless,2.000000,75148,http://nanowrimo.org/participants/nicaless,24,December 20,"Ursula Le Guin, J.K.","Classical, Musicals","Reading, Video Games, Blogging, Learning","San Francisco, CA",Consultant,NaN,NaN,1,1,2,"[2014, 2015]",[2014],[2014]
1,Rachel B. Moore,10.000000,528693,http://nanowrimo.org/participants/rachel-b-moore,NaN,NaN,"2666, Unaccustomed Earth, Exit Music, Crazy Lo...","Belle and Sebastian, Elliott Smith, PJ Harvey,...","Reading, volunteering, knitting, listening to ...",San Francisco,Writer/Office Monkey,Municipal Liaison,NaN,9,9,11,"[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...","[2005, 2006, 2007, 2008, 2009, 2010, 2011, 201...","[2007, 2008, 2009, 2010, 2011, 2012, 2013, 201..."
2,abookishbabe,1.000000,29299,http://nanowrimo.org/participants/abookishbabe,NaN,April 2,"Colleen Hoover, Veronica Roth, Jennifer Niven,...",Tori Kelley,"Reading (DUH), Day dreaming, Going to Disneyla...","Sacramento, CA",Optometry Assistant,NaN,NaN,1,0,1,[2015],[],[2015]
3,alexabexis,11.000000,554300,http://nanowrimo.org/participants/alexabexis,NaN,NaN,NaN,Three Goddesses playlist Florence + the Machin...,"drawing, reading, movies & TV shows, comics, p...",New York City,NaN,Municipal Liaison,https://www.classy.org/fundraise?fcid=498118,8,8,12,"[2004, 2005, 2006, 2007, 2008, 2009, 2010, 201...","[2004, 2005, 2008, 2011, 2012, 2013, 2014, 2015]","[2007, 2008, 2009, 2011, 2012, 2013, 2014, 2015]"
4,AllYellowFlowers,3.000000,41428,http://nanowrimo.org/participants/AllYellowFlo...,NaN,NaN,"Lolita, Jesus' Son, Ask the",the sound of the coffeemaker,cryptozoology,Allston,Bookstore Employee,NaN,NaN,0,0,3,"[2012, 2014, 2015]",[],[]
5,animalrza,3.000000,88131,http://nanowrimo.org/participants/animalrza,NaN,NaN,NaN,Miles Davis and the sort,"Photography, Writing, Yoga",San Francisco,NaN,NaN,NaN,0,0,4,"[2012, 2013, 2014, 2015]",[],[]
6,Cafenuit,3.000000,50145,http://nanowrimo.org/participants/Cafenuit,NaN,NaN,"Jhumpa Lahiri, Amy Hempel, Anais Nin, CE Pover...","Charlie Parker, Bill Evans, Miles Davis","writing, cooking, improv, running, yoga",San Francisco,Leadership & Development,NaN,NaN,1,1,1,[2015],[2015],[2015]
7,Carmelon,1.000000,18999,http://nanowrimo.org/participants/Carmelon,19,NaN,NaN,not sure yet,"drawing, painting, spoken word poetry, video g...",Bay Area - California,NaN,NaN,NaN,0,0,1,[2015],[],[]
8,carozy,4.000000,38981,http://nanowrimo.org/participants/carozy,NaN,NaN,"Radclyffe, John Scalzi, Q. Kelly, Louis",movie scores,"filmmaking, acting, reading, writing, biking",San Francisco,Office Manager,NaN,NaN,0,0,2,"[2012, 2015]",[],[]
9,chrisk0,10.000000,313332,http://nanowrimo.org/participants/chrisk0,NaN,NaN,"Isaac Asimov, Diane","Amanda Marshall, Great Big Sea, Dido",NaN,Southern Ontario,NaN,Municipal Liaison,NaN,5,5,5,"[2011, 2012, 2013, 2014, 2015]","[2011, 2012, 2013, 2014, 2015]","[2011, 2012, 2013, 2014, 2015]"
